<a href="https://colab.research.google.com/github/gimploo/suyati-ecom-project/blob/main/ML_CollabFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
books=pd.read_csv('Books.csv')
ratings = pd.read_csv("Ratings.csv")
book_df=pd.DataFrame(books,columns=['ISBN','Book'])
book_df.head()


In [ ]:
ratings['UserID'].value_counts()

In [ ]:
x = ratings['UserID'].value_counts() > 200
y = x[x].index
ratings = ratings[ratings['UserID'].isin(y)]
print(ratings)

In [ ]:
rating_with_books = ratings.merge(books, on='ISBN')
rating_with_books.head()

In [ ]:
number_rating = rating_with_books.groupby('Book')['BookRating'].count().reset_index()
number_rating.rename(columns= {'BookRating':'number_of_ratings'}, inplace=True)
final_rating = rating_with_books.merge(number_rating, on='Book')
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]
final_rating.drop_duplicates(['UserID','Book'], inplace=True)


In [ ]:
book_pivot = final_rating.pivot_table(columns='UserID', index='ISBN', values="BookRating")
book_pivot.fillna(0, inplace=True)
book_pivot.head()

In [ ]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)


In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

In [ ]:
distances, suggestions = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1, -1))

In [ ]:
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])